<p style="font-weight:bold;"> <span style="font-size: 36px"> Scenario Yield Curve Import Test </span> </p>

Uncomment the first three cells if you are debugging this test or running it separately.

In [0]:
//#!eval-notebook "../Initialization/InitSystemorphRefDataToMemory"

In [0]:
//#!eval-notebook "../Initialization/InitSystemorphUtils"

In [0]:
//var pathToFiles = "../Files/";
//await Import.InitializeDataNodesAsync(DataSource, pathToFiles, "CH");

In [0]:
DataSource.Reset(x => x.ResetType<RawVariable>().ResetType<IfrsVariable>().ResetType<YieldCurve>());

# Setting Constants

In [0]:
var EnableScenarioBackup = Scenarios.EnableScenario;

In [0]:
Scenarios.EnableScenario = true;

In [0]:
var argsBestEstimate = new ImportArgs ("CH", 2020, 12, Periodicity.Quarterly, null, ImportFormats.Cashflow);
var argsScenarioYieldCurve = new ImportArgs ("CH", 2020, 12, Periodicity.Quarterly, "YCUP1.0pct", ImportFormats.Cashflow);
var argsScenarioTransactionalData = new ImportArgs ("CH", 2020, 12, Periodicity.Quarterly, "SRUP1.0pct", ImportFormats.Cashflow);
var argsBestEstimateNextPeriod = new ImportArgs ("CH", 2021, 3, Periodicity.Quarterly, null, ImportFormats.Cashflow);
var argsScenarioYieldCurveNextPeriod = new ImportArgs ("CH", 2021, 3, Periodicity.Quarterly, "YCDW1.0pct", ImportFormats.Cashflow);

In [0]:
var estimateTypes = (await DataSource.Query<EstimateType>().ToArrayAsync());
var actualEstimateTypes = estimateTypes.Where(et => et.InputSource.Contains(InputSource.Actual)).Select(x => x.SystemName);
var cashflowEstimateTypes = estimateTypes.Where(et => et.InputSource.Contains(InputSource.Cashflow)).Select(x => x.SystemName);
var actualAndCashflowEstimateTypes = actualEstimateTypes.Intersect(cashflowEstimateTypes);
var onlyActualEstimateTypes = actualEstimateTypes.Except(cashflowEstimateTypes);
var onlyCashflowEstimateTypes = cashflowEstimateTypes.Except(actualEstimateTypes);

# Best Estimate Initialization

In [0]:
var basicYieldCurve = @"
@@Main
Year,Month,Scenario
2020,1,
@@YieldCurve
Currency,Values0,Values1,Values2,Values3,Name
USD,0.002,0.002,0.002,0.002,
EUR,0.002,0.002,0.002,0.002,NoDiscount";

In [0]:
await Import.FromString(basicYieldCurve).WithFormat(ImportFormats.YieldCurve).WithTarget(DataSource).ExecuteAsync()

In [0]:
var cashflowBestEstimateInit = @"
@@Main
ReportingNode,Year,Month,Scenario
CH,2020,12,
@@Cashflow
DataNode,AmountType,EstimateType,AocType,Novelty,AccidentYear,Values0,Values1,Values2,Values3,Values4,Values5,Values6,Values7,Values8,Values9,Values10,Values11,Values12,Values13,Values14,Values15,Values16,Values17,Values18,Values19,Values20,Values21,Values22,Values23
DT1.1,PR,BE,CL,C,,100,0,0,100,0,0,100,0,0,100,0,0,0,100,0,0,100,0,0,100,0,0,100,0
DT1.1,NIC,BE,CL,C,,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25
DT1.1,CU,P,CL,C,,-10,-10,-10,-10,-10,-10,-10,-10,-10,-10,-10,-10,-10,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-3
DT1.1,,RA,CL,C,,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5
DTR1.1,PR,BE,CL,C,,50,0,0,50,0,0,50,0,0,50,0,0,50,0,0,50,0,0,50,0,0,50,0,0
DTR1.1,NIC,BE,CL,C,,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5
DTR1.1,CU,P,CL,C,,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5
DTR1.1,,RA,CL,C,,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25";

In [0]:
var actualsBestEstimateInit = @"
@@Main
ReportingNode,Year,Month,Scenario
CH,2020,12,
@@Actual
DataNode,AocType,AmountType,EstimateType,AccidentYear,Value
DT1.1,CF,ACA,A,,-10
DT1.1,CF,AEA,A,,-5
DT1.1,CF,NIC,A,,-280
DT1.1,CF,PR,A,,400
DTR1.1,CF,PR,A,,-200
DTR1.1,CF,NIC,A,,140";

In [0]:
await Import.FromString(cashflowBestEstimateInit).WithFormat(ImportFormats.Cashflow).WithTarget(DataSource).ExecuteAsync()

In [0]:
await Import.FromString(actualsBestEstimateInit).WithFormat(ImportFormats.Actual).WithTarget(DataSource).ExecuteAsync()

In [0]:
await DataSource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(argsBestEstimate);
var ivsBestEstimateInit = await DataSource.Query<IfrsVariable>().ToArrayAsync();

In [0]:
DataSource.Partition.GetCurrentPartitions().Select(x => (x.Item1, x.Item2, x.Item1.GetType(), x.Item2.GetType()))

# Scenario Yield Curve Import

In [0]:
var yieldCurveScenario = @"
@@Main
Year,Month,Scenario
2020,12,YCUP1.0pct
@@YieldCurve
Currency,Values0,Values1,Values2,Values3,Name
EUR,0.108,0.108,0.118,0.119,
CHF,0.102,0.102,0.102,0.102,
USD,0.102,0.102,0.102,0.102,
EUR,0.002,0.002,0.002,0.002,3PCT";

In [0]:
await Import.FromString(yieldCurveScenario).WithFormat(ImportFormats.YieldCurve).WithTarget(DataSource).ExecuteAsync()

In [0]:
await DataSource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(argsScenarioYieldCurve);
var ivsScenarioYieldCurve = await DataSource.Query<IfrsVariable>().ToArrayAsync();

In [0]:
var diffs = ivsScenarioYieldCurve.Where(x => x.Values.Any(v => Math.Abs(v) > Precision)).Except(ivsBestEstimateInit, IfrsVariableComparer.Instance());
var actualVars = diffs.Where(x => onlyActualEstimateTypes.Contains(x.EstimateType)).ToArray();
var cashflowVars = diffs.Where(x => onlyCashflowEstimateTypes.Contains(x.EstimateType)).ToArray();

In [0]:
actualVars.Length.Should().Be(0);
cashflowVars.Length.Should().NotBe(0);

# Re-import of the same yield curve should not trigger calculations

In [0]:
var log = await Import.FromString(yieldCurveScenario).WithFormat(ImportFormats.YieldCurve).WithTarget(DataSource).ExecuteAsync();
log

In [0]:
((log.Warnings.FirstOrDefault()?.ToString().Remove(0,40).Remove(125,2)?? "") == Warning.VariablesAlreadyImported.GetMessage()).Should().Be(true);

In [0]:
await DataSource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(argsScenarioYieldCurve);
var ivsScenarioYieldCurveReimport = await DataSource.Query<IfrsVariable>().ToArrayAsync();

In [0]:
var diffs = ivsScenarioYieldCurve.Except(ivsScenarioYieldCurve, IfrsVariableComparer.Instance());

In [0]:
diffs.Count().Should().Be(0);

# Multiple Scenarios: re-import of Best Estimate yield curve

In this example, a new Cashflow for a different Scenario - SRUP1.0pct - is imported. 
The aim is to check how the Yield Curve importer behaves when a new Best Estimate Yield Curve is imported. 
We can identify the following two cases: 
 - a Scenario including a dedicated Yield Curve: only the Best Estimate variables are re-calculated;
 - a Scenario including only transactional data: re-calculation is needed to account for the updated Best Estimate Yield Curve. 

In [0]:
var cashflowScenario = @"
@@Main
ReportingNode,Year,Month,Scenario
CH,2020,12,SRUP1.0pct
@@Cashflow
DataNode,AmountType,EstimateType,AocType,Novelty,AccidentYear,Values0,Values1,Values2,Values3,Values4,Values5,Values6,Values7,Values8,Values9,Values10,Values11,Values12,Values13,Values14,Values15,Values16,Values17,Values18,Values19,Values20,Values21,Values22,Values23
DT1.1,PR,BE,CL,C,,110,0,0,110,0,0,110,0,0,110,0,0,0,110,0,0,110,0,0,110,0,0,110,0
DT1.1,NIC,BE,CL,C,,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5
DT1.1,,RA,CL,C,,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75";

In [0]:
await Import.FromString(cashflowScenario).WithFormat(ImportFormats.Cashflow).WithTarget(DataSource).ExecuteAsync()

In [0]:
await DataSource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(argsScenarioTransactionalData);
var ivsScenarioTransactionalData = await DataSource.Query<IfrsVariable>().ToArrayAsync();

In [0]:
var bestEstimateYieldCurve = @"
@@Main
Year,Month
2020,12
@@YieldCurve
Currency,Values0,Values1,Values2,Values3,Name
USD,0.006,0.004,0.004,0.005";

In [0]:
await Import.FromString(bestEstimateYieldCurve).WithFormat(ImportFormats.YieldCurve).WithTarget(DataSource).ExecuteAsync()

## Best Estimate variables are re-calculated

In [0]:
await DataSource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(argsBestEstimate);
var ivsBestEstimateUpdated = await DataSource.Query<IfrsVariable>().ToArrayAsync();

In [0]:
var diffs = ivsBestEstimateUpdated.Except(ivsBestEstimateInit, IfrsVariableComparer.Instance());

In [0]:
diffs.Count().Should().NotBe(0);

## Scenario with dedicated Yield Curve does not change

In [0]:
await DataSource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(argsScenarioYieldCurve);
var ivsScenarioYieldCurveUpdated = await DataSource.Query<IfrsVariable>().ToArrayAsync();

In [0]:
var diffs = ivsScenarioYieldCurveUpdated.Where(x => x.Values.Any(v => Math.Abs(v) > Precision)).Except(ivsScenarioYieldCurve, IfrsVariableComparer.Instance());

In [0]:
diffs.Count().Should().Be(0);

## Scenario including Transactional Data solely is re-calculated

In [0]:
await DataSource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(argsScenarioTransactionalData);
var ivsScenarioTransactionalDataUpdated = await DataSource.Query<IfrsVariable>().ToArrayAsync();

In [0]:
var diffs = ivsScenarioTransactionalDataUpdated.Where(x => x.Values.Any(v => Math.Abs(v) > Precision)).Except(ivsScenarioTransactionalData, IfrsVariableComparer.Instance());

In [0]:
diffs.Count().Should().NotBe(0);

# Influence on the future: import of yield curve for previous month should not update current results

In [0]:
var bestEstimateYieldCurvePast = @"
@@Main
Year,Month
2020,6
@@YieldCurve
Currency,Values0,Values1,Values2,Values3
USD,0.123,0.123,0.123,0.123";

In [0]:
await Import.FromString(bestEstimateYieldCurvePast).WithFormat(ImportFormats.YieldCurve).WithTarget(DataSource).ExecuteAsync()

## Best Estimate should not change

In [0]:
await DataSource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(argsBestEstimate);
var ivsBestEstimateImportPast = await DataSource.Query<IfrsVariable>().ToArrayAsync();

In [0]:
var diffs = ivsBestEstimateImportPast.Except(ivsBestEstimateUpdated, IfrsVariableComparer.Instance());

In [0]:
diffs.Count().Should().Be(0);

## Scenario should not change

In [0]:
await DataSource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(argsScenarioYieldCurve);
var ivsScenarioYieldCurveImportPast = await DataSource.Query<IfrsVariable>().ToArrayAsync();

In [0]:
var diffs = ivsScenarioYieldCurveImportPast.Except(ivsScenarioYieldCurveUpdated, IfrsVariableComparer.Instance());

In [0]:
diffs.Count().Should().Be(0);

# Next period imports

In [0]:
var cashflowBestEstimateNextPeriod = @"
@@Main
ReportingNode,Year,Month,Scenario
CH,2021,3
@@Cashflow
DataNode,AmountType,EstimateType,AocType,Novelty,AccidentYear,Values0,Values1,Values2,Values3,Values4,Values5,Values6,Values7,Values8,Values9,Values10,Values11,Values12,Values13,Values14,Values15,Values16,Values17,Values18,Values19,Values20,Values21,Values22,Values23
DT1.1,PR,BE,CL,C,,80,0,0,80,0,0,80,0,0,80,0,0,0,80,0,0,80,0,0,80,0,0,80,0
DT1.1,CU,P,CL,C,,1,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,1,0
DT1.1,NIC,BE,CL,C,,-22.3,-22.3,-22.3,-22.3,-22.3,-22.3,-22.3,-22.3,-22.3,-22.3,-22.3,-22.3,-22.3,-22.3,-22.3,-22.3,-22.3,-22.3,-22.3,-22.3,-22.3,-22.3,-22.3,-22.3
DT1.1,,RA,CL,C,,-2.40,-2.40,-2.40,-2.40,-2.40,-2.40,-2.40,-2.40,-2.40,-2.40,-2.40,-2.40,-2.40,-2.40,-2.40,-2.40,-2.40,-2.40,-2.40,-2.40,-2.40,-2.40,-2.40,-2.40
DTR1.1,PR,BE,CL,C,,50,0,0,50,0,0,50,0,0,50,0,0,50,0,0,50,0,0,50,0,0,50,0,0
DTR1.1,NIC,BE,CL,C,,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5
DTR1.1,CU,P,CL,C,,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5
DTR1.1,,RA,CL,C,,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25";

In [0]:
var yieldCurveScenarioNextPeriod = @"
@@Main
Year,Month,Scenario
2021,3,YCDW1.0pct
@@YieldCurve
Currency,Values0,Values1,Values2,Values3
USD,0.007,0.004,0.004,0.005";

In [0]:
await Import.FromString(cashflowBestEstimateNextPeriod).WithFormat(ImportFormats.Cashflow).WithTarget(DataSource).ExecuteAsync()

In [0]:
await Import.FromString(yieldCurveScenarioNextPeriod).WithFormat(ImportFormats.YieldCurve).WithTarget(DataSource).ExecuteAsync()

## Results with economic basis locked in basis should not be affected

In [0]:
await DataSource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(argsBestEstimateNextPeriod);
var ivsBestEstimateNextPeriod = await DataSource.Query<IfrsVariable>().ToArrayAsync();

In [0]:
await DataSource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(argsScenarioYieldCurveNextPeriod);
var ivsScenarioYieldCurveNextPeriod = await DataSource.Query<IfrsVariable>().ToArrayAsync();

In [0]:
var diffs = ivsScenarioYieldCurveNextPeriod.Except(ivsBestEstimateNextPeriod, IfrsVariableComparer.Instance());

In [0]:
diffs.Where(x => x.EconomicBasis == EconomicBases.C).Count().Should().NotBe(0);
diffs.Where(x => x.EconomicBasis == EconomicBases.L).Count().Should().Be(0);

# Restore Constants

In [0]:
Scenarios.EnableScenario = EnableScenarioBackup;